<a href="https://colab.research.google.com/github/jeeva-18/Deep-learning-projects/blob/main/101_foodvisionmodel_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow_datasets as tfds

In [ ]:
list_datasets = tfds.list_builders()
print('food101' in list_datasets) # checking for our dataset presence

True


In [ ]:
# it will download the data into our sytems if you are using your local computer it will download lots of data  
(train_data,test_data),df_info = tfds.load('food101',
                                           split=['train','validation'], # in our case we have training and validation but some dataset have testing too
                                           shuffle_files=True, # we are shuffle our files to some randomness in our data 
                                           as_supervised=True, # beacuse our dataset is supervised 
                                           with_info=True) # getting metadata about our dataset

In [ ]:
#features of the dataset
df_info.features

In [ ]:
# getting the class names 

class_names = df_info.features['label'].names
class_names[:10]

In [ ]:
# lets see some info about images and labels
for img ,labels in train_data.take(1): # .take method will take one sample from the data and every time we rn these we get random values because we set shuffle = True 
  print(f'''
  image shape: {img.shape}
  image dtype: {img.dtype}
  label : {labels}
  label name: {class_names[labels.numpy()]}
  '''
  )

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

plt.figure(figsize=(15,15))
for i,(img ,labels) in enumerate(train_data.take(9)):
  plt.subplot(3,3,i+1)
  plt.imshow(img)
  plt.title(class_names[labels.numpy()])
  plt.axis(False)

In [ ]:
# lets create a function that will  preprocess our data

def preprocess_image_data(images,labels,img_size=(224,224)):
  '''
  This function will reduce image size and change dtype int float32

  '''
  image  = tf.image.resize(images,img_size)

  return tf.cast(image,dtype=tf.float32),labels

In [ ]:
# lets check our function 

prep_img ,label = preprocess_image_data(img,labels)

print(f'image_shape : {prep_img.shape} image_dtype : {prep_img.dtype} ')

In [ ]:
# we are going to use map method to map our preprocess_image_data function for preprocessing

train_data = train_data.map(map_func=preprocess_image_data,num_parallel_calls=tf.data.AUTOTUNE)

# making into batches for training data

train_data = train_data.shuffle(buffer_size=1000).batch(batch_size=32).prefetch(buffer_size=tf.data.AUTOTUNE)

# preprocess and making batches for testing data

test_data  = test_data.map(map_func=preprocess_image_data,num_parallel_calls=tf.data.AUTOTUNE).batch(32).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
train_data,test_data

In [ ]:
import os 
import datetime
# creating the tensoboard callback
def create_tensorboard_callback(dir_name,experiment):
  date_time = datetime.datetime.now().strftime('%Y/%m/%d:%H-%M-%S')
  path = os.path.join(dir_name,experiment,date_time)
  return tf.keras.callbacks.TensorBoard(log_dir=path)

In [ ]:
# model check point callback

checkpoint_path = 'model_checkpoints/cp.cpkt'

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                         monitor='val_accuracy',
                                                         save_best_only=True,
                                                         save_weights_only=True,
                                                         verbose=0)

In [ ]:
# start by importing it 
from tensorflow.keras import mixed_precision

mixed_precision.set_global_policy('mixed_float16') # this will do the job

In [ ]:
!nvidia-smi -L  # offcourse mixed precision will only works on the gpu with computing capability of above 7.0, likely our's tesla t4 got 7.5 :)  

In [ ]:
mixed_precision.global_policy() # you can use this we utilize large amount of data

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing 

# we are going t use the transfer learning - feature extraction

input_shape = (224,224,3)

# our base model will be EfficientNetB0
base_model = tf.keras.applications.EfficientNetV2B0(include_top=False)
base_model.trainable = False

input = layers.Input(shape=input_shape,name='input_layer')

# data_augmentation = tf.keras.Sequential([ # this is for  data augmentation
#     preprocessing.RandomRotation(20),
#     preprocessing.RandomZoom(0.2),
#     preprocessing.RandomFlip(mode='horizontal'), 
#     preprocessing.RandomWidth(0.2),
#     preprocessing.RandomHeight(0.2),
#     preprocessing.Rescaling(1/255.)      
# ])(input)

x = base_model(input)

x = layers.GlobalAveragePooling2D(name='globalAvgPooling_layer')(x)

x = layers.Dense(101,name='output_layer')(x)

output = layers.Activation(activation='softmax',dtype=tf.float32,name='prediction')(x)

model = tf.keras.Model(input,output,name='food_vision_model')

model.compile(loss='sparse_categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
# mixed precision works or not 

for layer in model.layers:
  print(layer.name,layer.trainable,layer.dtype,layer.dtype_policy)

In [ ]:
# lets check it in our base model

for layer in model.layers[1].layers[:20]:
  print(layer.name,layer.trainable,layer.dtype,layer.dtype_policy)

In [ ]:
feature_extraction_history = model.fit(train_data,
                                       epochs=5,
                                       steps_per_epoch=len(train_data),
                                       validation_data=test_data,
                                       validation_steps=int(.15*len(test_data)),
                                       callbacks=[create_tensorboard_callback('models','FX_efficientnet0'),checkpoint_callback])

In [ ]:
fx_results = model.evaluate(test_data)
fx_results

In [ ]:
# we are going to create a plot of history of the model
import matplotlib.pyplot as plt


def plot_history(history):
  """
  this will give you the plot of the history instance
  """
  acc = history.history['accuracy']
  val_acc = history.history['val_accuracy']

  loss = history.history['loss']
  val_loss = history.history['val_loss']

  # plot accuracy 
  plt.figure()
  plt.plot(acc,label='Training')
  plt.plot(val_acc,label='validation')
  plt.title('Accuracy scores')
  plt.legend()
  #plot loss
  plt.figure()
  plt.plot(loss,label='Training')
  plt.plot(val_loss,label='validation')
  plt.title('Losses')
  plt.legend()

In [ ]:
plot_history(feature_extraction_history)

In [ ]:
# lets make it up

base_model.trainable = True


for layer in base_model.layers[:-20]:
  layer.trainable = False

In [ ]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(1e-4), # we are reducing the learning rate so pretrained weights does not change too much 
              metrics=['accuracy'])

In [ ]:
for layer in model.layers:
  print(layer.name,layer.trainable)

In [ ]:
for num,layer in enumerate(model.layers[1].layers):
  print(num,layer.name,layer.trainable)

In [ ]:
# Setup EarlyStopping callback to stop training if model's val_loss doesn't improve for 3 epochs
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", # watch the val loss metric
                                                  patience=3) # if val loss decreases for 3 epochs in a row, stop training

# Create ModelCheckpoint callback to save best model during fine-tuning
checkpoint_path = "fine_tune_checkpoints/"
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                      save_best_only=True,
                                                      monitor="val_loss")

In [ ]:
# creating the callbacks to reduce learning rate of the model
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss",  
                                                 factor=0.2, # multiply the learning rate by 0.2 (reduce by 5x)
                                                 patience=2,
                                                 verbose=1, # print out when learning rate goes down 
                                                 min_lr=1e-7)

In [ ]:
best_model = model.fit(train_data,
                       epochs=100,
                       steps_per_epoch=len(train_data),
                       validation_data=test_data,
                       validation_steps=int(.15 * len(test_data)),
                       callbacks=[create_tensorboard_callback('models','best_fine_effb0'),
                                  early_stopping,reduce_lr,model_checkpoint])

In [ ]:
best_results = model.evaluate(test_data)
best_results

In [ ]:
plot_history(best_model)